In [ ]:
import random
import json
from tqdm import tqdm
import unicodedata

import torch

# local modules
from ner_tokenizer_bio import NER_tokenizer_BIO
from bert_for_token_classification_pl import BertForTokenClassification_pl

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [ ]:
best_model_path='./model/epoch=4-step=660.ckpt'

In [ ]:
# 性能評価
model = BertForTokenClassification_pl.load_from_checkpoint(
    best_model_path
)
bert_tc = model.bert_tc.cuda()

In [ ]:
# スロットの種類数 (COL=1, COLLTDEV=2, LOC=3, ONOFFDEV=4, OPENABLE=5, TEMPDEV=6, TEMPERTURE_NUM=7, THMDEV=8)
NUM_ENTITY_TYPE = 8

# トークナイザのロード
# 固有表現のカテゴリーの数`num_entity_type`を入力に入れる必要がある。
tokenizer = NER_tokenizer_BIO.from_pretrained(
    MODEL_NAME,
    num_entity_type=NUM_ENTITY_TYPE
)

In [ ]:
# 個別に実行
entities           = [] # 正解の固有表現
entities_predicted = [] # 抽出された固有表現

text = unicodedata.normalize('NFKC', '会議室にある黄色い電灯の火を点灯してくださいな')

encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
)
encoding = { k: v.cuda() for k, v in encoding.items() } 

with torch.no_grad():
    output = bert_tc(**encoding)
    # tuple の 2 番めには tuple(logits_intent, logits_slot) が入っている
    logits_tuple  = output[1]
    logits_intent = logits_tuple[0]
    logits_slot = logits_tuple[1][0]
    scores_intent = logits_intent.cpu().numpy()
    scores_slots  = logits_slot.cpu().numpy().tolist()

# Intent 分類スコアを Intent に変換する
intent = scores_intent.argmax(-1)[0]
# Slot 分類スコアを固有表現に変換する
entities_predicted = tokenizer.convert_bert_output_to_entities(
    text, scores_slots, spans
)

print("入力",text)
print("予測 intent  :", intent)
print("予測 entities:", json.dumps(entities_predicted, indent=2, ensure_ascii=False))